In [ ]:
%matplotlib inline


Neural Networks
===============

使用torch.nn包来构建神经网络。

上一讲已经讲过了``autograd``，``nn``包依赖``autograd``包来定义模型并求导。
一个``nn.Module``包含各个层和一个``forward(input)``方法，该方法返回``output``。



例如：

![](https://pytorch.org/tutorials/_images/mnist.png)

它是一个简单的前馈神经网络，它接受一个输入，然后一层接着一层地传递，最后输出计算的结果。

神经网络的典型训练过程如下：

1. 定义包含一些可学习的参数(或者叫权重)神经网络模型； 
2. 在数据集上迭代； 
3. 通过神经网络处理输入； 
4. 计算损失(输出结果和正确值的差值大小)；
5. 将梯度反向传播回网络的参数； 
6. 更新网络的参数，主要使用如下简单的更新原则： 
``weight = weight - learning_rate * gradient``

  

定义网络
------------------

开始定义一个网络：



In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(2, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        # x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        a = self.conv1(x)
        print(a.size())
        b = F.relu(a)
        print(b.size())
        x = F.max_pool2d(b, (2, 2))
        print(x.size())
        # # If the size is a square you can only specify a single number
        # x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # print(x.size())
        # x = x.view(-1, self.num_flat_features(x))
        # # x = x.view(-1, 200)
        # print(x.size())
        # x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x))
        # x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        print(num_features)
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(2, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


在模型中必须要定义 ``forward`` 函数，``backward``
函数（用来计算梯度）会被``autograd``自动创建。
可以在 ``forward`` 函数中使用任何针对 Tensor 的操作。

 ``net.parameters()``返回可被学习的参数（权重）列表和值



In [2]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight
print(params[1].size())  # conv1's .weight
print(params[2].size())  # conv1's .weight
print(params[3].size())  # conv1's .weight
print(params[4].size())  # conv1's .weight
print(params[5].size())  # conv1's .weight
print(net._version)

10
torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 400])
torch.Size([120])
1


测试随机输入32×32。
注：这个网络（LeNet）期望的输入大小是32×32，如果使用MNIST数据集来训练这个网络，请把图片大小重新调整到32×32。



In [10]:
input = torch.randn(1, 2, 32, 32)
out = net(input)
print(out)

torch.Size([1, 6, 28, 28])
torch.Size([1, 6, 28, 28])
torch.Size([1, 6, 14, 14])
tensor([[[[0.7261, 0.6407, 1.0906,  ..., 1.0840, 0.7874, 0.6428],
          [0.8036, 0.4795, 0.0000,  ..., 0.9495, 0.5942, 0.3541],
          [1.2193, 0.7010, 0.5772,  ..., 0.0835, 0.1956, 0.8871],
          ...,
          [0.7455, 0.7804, 1.0367,  ..., 0.1396, 1.0607, 0.2267],
          [0.5692, 0.4201, 0.8629,  ..., 0.2429, 0.7983, 0.0000],
          [0.5114, 0.0000, 0.2079,  ..., 0.9539, 0.9879, 0.2722]],

         [[0.4196, 0.0235, 0.8354,  ..., 1.0783, 0.1836, 0.5771],
          [0.4151, 0.7943, 1.0452,  ..., 0.4610, 0.7724, 1.0821],
          [0.2803, 0.8805, 1.0858,  ..., 0.1035, 0.9998, 0.7724],
          ...,
          [1.1401, 1.0061, 0.8896,  ..., 0.6259, 0.1779, 1.0371],
          [1.0320, 0.5614, 0.5005,  ..., 0.3969, 1.1002, 1.3909],
          [0.5669, 0.7030, 1.0018,  ..., 0.4878, 0.7614, 0.8180]],

         [[0.0629, 0.4874, 0.1219,  ..., 0.7211, 0.5005, 0.4643],
          [0.6833, 0.1275, 

In [7]:
net.zero_grad()
out.backward(torch.randn(1, 10))

<div class="alert alert-info"><h4>Note</h4><p>``torch.nn`` 只支持小批量输入。整个 ``torch.nn``
包都只支持小批量样本，而不支持单个样本。

    例如，``nn.Conv2d`` 接受一个4维的张量，
    ``每一维分别是sSamples * nChannels * Height * Width（样本数*通道数*高*宽）``。

    如果你有单个样本，只需使用 ``input.unsqueeze(0)`` 来添加其它的维数</p></div>

在继续之前，我们回顾一下到目前为止用到的类。

**回顾:**
  -  ``torch.Tensor``：一个用过自动调用 ``backward()``实现支持自动梯度计算的 *多维数组* ，
      并且保存关于这个向量的*梯度* w.r.t.
  -  ``nn.Module``：神经网络模块。封装参数、移动到GPU上运行、导出、加载等。
  -  ``nn.Parameter``：一种变量，当把它赋值给一个``Module``时，被 *自动* 地注册为一个参数。
  -  ``autograd.Function``：实现一个自动求导操作的前向和反向定义，每个变量操作至少创建一个函数节点，每一个``Tensor``的操作都回创建一个接到创建``Tensor``和 *编码其历史* 的函数的``Function``节点。

**重点如下：**
  -  定义一个网络
  -  处理输入，调用backword

**还剩：**
  -  计算损失
  -  更新网络权重

损失函数
-------------
一个损失函数接受一对 (output, target) 作为输入，计算一个值来估计网络的输出和目标值相差多少。

***译者注：output为网络的输出，target为实际值***

nn包中有很多不同的[损失函数](https://pytorch.org/docs/nn.html#loss-functions)。
``nn.MSELoss``是一个比较简单的损失函数，它计算输出和目标间的**均方误差**，
例如：



In [3]:
output = net(input)
print(output.size())
target = torch.randn(10)  # 随机值作为样例
print(target.size())
target = target.view(1, -1)  # 使target和output的shape相同
print(target.size())
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

torch.Size([1, 6, 14, 14])
torch.Size([1, 16, 5, 5])
400
torch.Size([1, 400])
torch.Size([1, 10])
torch.Size([10])
torch.Size([1, 10])
tensor(1.0138, grad_fn=<MseLossBackward0>)


现在，如果在反向过程中跟随``loss`` ， 使用它的
``.grad_fn`` 属性，将看到如下所示的计算图。

::

    input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
          -> view -> linear -> relu -> linear -> relu -> linear
          -> MSELoss
          -> loss

所以，当我们调用 ``loss.backward()``时,整张计算图都会
根据loss进行微分，而且图中所有设置为``requires_grad=True``的张量
将会拥有一个随着梯度累积的``.grad`` 张量。

为了说明，让我们向后退几步:



In [9]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

反向传播
--------
调用loss.backward()获得反向传播的误差。

但是在调用前需要清除已存在的梯度，否则梯度将被累加到已存在的梯度。

现在，我们将调用loss.backward()，并查看conv1层的偏差（bias）项在反向传播前后的梯度。




In [10]:
net.zero_grad()     # 清除梯度

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0051,  0.0042,  0.0026,  0.0152, -0.0040, -0.0036])


如何使用损失函数

**稍后阅读：**

  `nn`包，包含了各种用来构成深度神经网络构建块的模块和损失函数，完整的文档请查看[here](https://pytorch.org/docs/nn)。

**剩下的最后一件事:**

  - 新网络的权重

更新权重
------------------
在实践中最简单的权重更新规则是随机梯度下降（SGD）：

     ``weight = weight - learning_rate * gradient``

我们可以使用简单的Python代码实现这个规则：

```python

learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)
```
但是当使用神经网络是想要使用各种不同的更新规则时，比如SGD、Nesterov-SGD、Adam、RMSPROP等，PyTorch中构建了一个包``torch.optim``实现了所有的这些规则。
使用它们非常简单：


In [11]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

.. 注意::
    
      观察如何使用``optimizer.zero_grad()``手动将梯度缓冲区设置为零。
      这是因为梯度是按Backprop部分中的说明累积的。

